A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [19]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout,Flatten
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [5]:
len(labels)

78

In [6]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [7]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [8]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [9]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [10]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [12]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [13]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [14]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [15]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [37]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    flat = Flatten()(embd_sentence)

    hidden1=Dense(400,activation="relu")(flat)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute_7 (Permute)          (None, 300, 300)          0         
_________________________________________________________________
lambda_13 (Lambda)           (None, 300, 300, 1)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 90000)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 400)               36000400  
_________________________________________________________________
dense_46 (Dense)             (None, 300)   

In [38]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
405/405 [==============================] - 86s 210ms/step - loss: 0.2690 - accuracy: 0.0477 - val_loss: 0.0997 - val_accuracy: 0.1709

Epoch 00001: val_loss improved from inf to 0.09966, saving model to best_cnn_4fc.h5
Epoch 2/250
405/405 [==============================] - 87s 214ms/step - loss: 0.0899 - accuracy: 0.2279 - val_loss: 0.0896 - val_accuracy: 0.2558

Epoch 00002: val_loss improved from 0.09966 to 0.08955, saving model to best_cnn_4fc.h5
Epoch 3/250
405/405 [==============================] - 86s 213ms/step - loss: 0.0775 - accuracy: 0.3289 - val_loss: 0.0844 - val_accuracy: 0.2866

Epoch 00003: val_loss improved from 0.08955 to 0.08439, saving model to best_cnn_4fc.h5
Epoch 4/250
405/405 [==============================] - 87s 215ms/step - loss: 0.0668 - accuracy: 0.4167 - val_loss: 0.0821 - val_accuracy: 0.3161

Epoch 00004: val_loss improved from 0.08439 to 0.08209, saving model to best_cnn_4fc.h5
Epoch 5/250
405/405 [==============================] - 88s 217ms

# ***If the model is generated before:***

In [40]:
model = load_model('CNN_1_just_accuracy_No_CNN.h5')
# model.save('CNN_1_just_accuracy_No_CNN.h5')

In [41]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [42]:
print(pred[0])#example

[3.3869921e-11 9.6704892e-12 1.8908304e-08 4.5301363e-07 2.8852281e-09
 3.5039611e-09 1.7123689e-12 7.7456404e-16 2.9307587e-09 1.7429190e-11
 1.4807677e-12 5.3584355e-09 1.3996488e-12 9.6152938e-12 5.1754699e-12
 2.8132252e-09 2.0918808e-08 5.2773891e-07 2.2860956e-11 1.6879296e-05
 5.8745250e-13 7.1617784e-10 5.5903524e-12 1.0379296e-07 1.3286935e-09
 1.8041689e-10 1.6563564e-10 6.9157534e-07 1.6225803e-09 3.6777923e-08
 2.2010502e-06 4.2391541e-13 1.5819519e-07 2.8360482e-08 1.9223273e-06
 1.2363119e-09 1.3273284e-07 7.7145032e-08 7.9126685e-06 2.3939396e-07
 8.8003962e-12 2.5060316e-09 2.6533503e-10 7.1736491e-07 1.5345391e-08
 5.8912559e-09 1.4978589e-09 2.6526760e-11 1.3031061e-09 5.2778472e-05
 3.1196039e-09 1.5279947e-07 1.0769791e-05 3.0939871e-09 2.3866550e-10
 9.9283682e-07 1.6224442e-07 6.1779683e-06 6.6702664e-03 1.0310690e-07
 9.9640918e-01 5.8602264e-08 5.6702004e-10 7.2856883e-16 9.4592554e-12
 2.0710184e-08 1.3190937e-08 2.7130654e-06 3.6076411e-08 1.6781257e-11
 5.050

In [43]:
y_pred=[]
measure = .23#9*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [44]:
3*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))

0.20749984122812748

In [45]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


accuracy= 0.19480519480519481
              precision    recall  f1-score   support

           0       0.32      0.17      0.22       119
           1       0.17      0.11      0.13       159
           2       0.35      0.26      0.30       151
           3       0.00      0.00      0.00        16
           4       0.97      0.82      0.89        68
           5       0.00      0.00      0.00        11
           6       0.30      0.23      0.26       163
           7       0.67      0.11      0.19        36
           8       0.00      0.00      0.00        24
           9       0.00      0.00      0.00        91
          10       0.28      0.18      0.22       140
          11       0.97      0.92      0.95        76
          12       0.99      0.98      0.98       126
          13       0.00      0.00      0.00        14
          14       0.00      0.00      0.00        11
          15       0.19      0.09      0.12       149
          16       0.62      0.41      0.49       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
